<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8 Assistants API —快速開發助理應用程式

## 8-1 什麼是 Assistants-API

### Assistants API 的基本元件

In [ ]:
!pip install openai rich googlesearch-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=83af38f683c0eec6630116409c9ec62b72f5d608060664a7a518067069bdc308
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
from google.colab import userdata
from rich import print as pprint
from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

### 建立 Assistant

In [ ]:
assistant = client.beta.assistants.create(
    name="運動達人",
    instructions="你是一位熱愛運動的達人, 具有各種與運動相關的知識",
    model="gpt-3.5-turbo-1106"
)
pprint(assistant)

Assistant(
    id='asst_EHR4omyzzojIfQ8spmohkvaB',
    created_at=1703569838,
    description=None,
    file_ids=[],
    instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
    metadata={},
    model='gpt-3.5-turbo-1106',
    name='運動達人',
    object='assistant',
    tools=[]
)

### 建立 Thread 與 Message

In [ ]:
thread = client.beta.threads.create()
pprint(thread)

Thread(id='thread_1r6m1Agm9WkjWk9Eif5o3NOK', created_at=1703569840, metadata={}, object='thread')

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="走路一小時相當於消耗多少熱量？"
)
pprint(message)

ThreadMessage(
    id='msg_aJVCCSuuU3ZutHzJkSx80BxQ',
    assistant_id=None,
    content=[MessageContentText(text=Text(annotations=[], value='走路一小時相當於消耗多少熱量？'), type='text')],
    created_at=1703569842,
    file_ids=[],
    metadata={},
    object='thread.message',
    role='user',
    run_id=None,
    thread_id='thread_1r6m1Agm9WkjWk9Eif5o3NOK'
)

### 建立 Run 執行任務

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
pprint(run)

Run(
    id='run_lHXzcCHn8NSObn8ZtOudM6AA',
    assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
    cancelled_at=None,
    completed_at=None,
    created_at=1703569843,
    expires_at=1703570443,
    failed_at=None,
    file_ids=[],
    instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
    last_error=None,
    metadata={},
    model='gpt-3.5-turbo-1106',
    object='thread.run',
    required_action=None,
    started_at=None,
    status='queued',
    thread_id='thread_1r6m1Agm9WkjWk9Eif5o3NOK',
    tools=[]
)

In [ ]:
import time

def wait_on_run(run):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=run.thread_id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run
run = wait_on_run(run)
pprint(run)

### 顯示討論串內的所有訊息

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
pprint(messages)

SyncCursorPage[ThreadMessage](
    data=[
        ThreadMessage(
            id='msg_LAJdOG3zD2dcPp8jwRw7hZci',
            assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
            content=[
                MessageContentText(
                    text=Text(
                        annotations=[],
                        value='消耗的熱量取決於走路的速度以及個人的體重和身體情況。一般而言，以5公里/小時的速度行走
一小時，一個65公斤的人大約會消耗250-300卡路里的熱量。然而，這些數字僅供參考，實際的消耗量還會受到其他因素的影響，例
如地形、風向、走路時是否攜帶負重等。'
                    ),
                    type='text'
                )
            ],
            created_at=1703569844,
            file_ids=[],
            metadata={},
            object='thread.message',
            role='assistant',
            run_id='run_lHXzcCHn8NSObn8ZtOudM6AA',
            thread_id='thread_1r6m1Agm9WkjWk9Eif5o3NOK'
        ),
        ThreadMessage(
            id='msg_aJVCCSuuU3ZutHzJkSx80BxQ',
            assistant_id=None,
            content=[
                MessageContentText(
                    text=Text(annotations=[], value='走路一小時相當於消耗多少熱量？'),
                    type='text'
                )
            ],
            created_at=1703569842,
            file_ids=[],
            metadata={},
            object='thread.message',
            role='user',
            run_id=None,
            thread_id='thread_1r6m1Agm9WkjWk9Eif5o3NOK'
        )
    ],
    object='list',
    first_id='msg_LAJdOG3zD2dcPp8jwRw7hZci',
    last_id='msg_aJVCCSuuU3ZutHzJkSx80BxQ',
    has_more=False
)

In [ ]:
for part in messages:
    print(f'{part.role}：{part.content[0].text.value}')

assistant：消耗的熱量取決於走路的速度以及個人的體重和身體情況。一般而言，以5公里/小時的速度行走一小時，一個65公斤的人大約會消耗250-300卡路里的熱量。然而，這些數字僅供參考，實際的消耗量還會受到其他因素的影響，例如地形、風向、走路時是否攜帶負重等。
user：走路一小時相當於消耗多少熱量？


## 8-2 Assistants API 物件用法

### 建立新討論串直接執行

In [ ]:
run = client.beta.threads.create_and_run(
  assistant_id=assistant.id,
  thread={
    "messages": [
      {"role": "user",
       "content": "當小腿抽筋時最適當的處理是?, 用繁體中文回答"}
    ],
  }
)
pprint(run)
run = wait_on_run(run)
messages = client.beta.threads.messages.list(thread_id=run.thread_id)
for part in messages:
    print(f'{part.role}：{part.content[0].text.value}')

Run(
    id='run_zxEUhKJrJNJMQk10w9xx2MmV',
    assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
    cancelled_at=None,
    completed_at=None,
    created_at=1703570052,
    expires_at=1703570652,
    failed_at=None,
    file_ids=[],
    instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
    last_error=None,
    metadata={},
    model='gpt-3.5-turbo-1106',
    object='thread.run',
    required_action=None,
    started_at=None,
    status='queued',
    thread_id='thread_KtqImOv443wCrhGngaLblots',
    tools=[]
)

assistant：當小腿抽筋時，最好的處理方法是停止運動，放鬆緊繃的肌肉，輕輕按摩或拉伸受影響的區域，同時進行深呼吸放鬆身體。另外，可透過按摩或應用熱敷來緩解疼痛感。若疼痛持續或頻繁發生，建議尋求專業醫療建議。
user：當小腿抽筋時最適當的處理是?, 用繁體中文回答


In [ ]:
new_thread = client.beta.threads.retrieve(run.thread_id)
pprint(new_thread)

Thread(id='thread_KtqImOv443wCrhGngaLblots', created_at=1703570052, metadata={}, object='thread')

### 建立對話函式

In [ ]:
def input_and_run(input, thread_id, assistant_id):
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=input
    )
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )

    return message, run

In [ ]:
message, run = input_and_run("燙傷時第一步該怎麼做？",
                             new_thread.id, assistant.id)
wait_on_run(run)
messages = client.beta.threads.messages.list(thread_id=run.thread_id)
for part in messages:
    print(f'{part.role}：{part.content[0].text.value}')

assistant：燙傷時，第一步是立即將受傷部位放置在冷水下沖洗，持續約 10 至 15 分鐘，以降低受傷部位的溫度。切記不要使用冰塊直接觸碰受傷部位，以免造成更多傷害。冷水可以有效減輕疼痛、減少紅腫和防止燙傷加重。如果是擦傷燙傷的情況，可以輕輕擦拭傷口後再進行冷水沖洗。如果傷口擦傷面積較大或有疑慮，建議儘快就醫處理。
user：燙傷時第一步該怎麼做？
assistant：當小腿抽筋時，最好的處理方法是停止運動，放鬆緊繃的肌肉，輕輕按摩或拉伸受影響的區域，同時進行深呼吸放鬆身體。另外，可透過按摩或應用熱敷來緩解疼痛感。若疼痛持續或頻繁發生，建議尋求專業醫療建議。
user：當小腿抽筋時最適當的處理是?, 用繁體中文回答


### 取消執行

In [ ]:
message, run = input_and_run('蒙娜麗莎是誰的作品？',
                             new_thread.id, assistant.id)

run = client.beta.threads.runs.cancel(
  thread_id=new_thread.id,
  run_id=run.id
)
print(f"取消中：{run.status}")
while run.status == "cancelling":
    run = client.beta.threads.runs.retrieve(
            thread_id=new_thread.id,
            run_id=run.id,
        )
    if run.status != "cancelling":
        print(f"已取消：{run.status}")
        break

取消中：cancelling
已取消：cancelled


### 檢視執行記錄

In [ ]:
runs = client.beta.threads.runs.list(
  new_thread.id
)
pprint(runs)

SyncCursorPage[Run](
    data=[
        Run(
            id='run_oNEohwsNB800aPqcX0XmzrV0',
            assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
            cancelled_at=1703570065,
            completed_at=None,
            created_at=1703570063,
            expires_at=None,
            failed_at=None,
            file_ids=[],
            instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
            last_error=None,
            metadata={},
            model='gpt-3.5-turbo-1106',
            object='thread.run',
            required_action=None,
            started_at=1703570064,
            status='cancelled',
            thread_id='thread_KtqImOv443wCrhGngaLblots',
            tools=[]
        ),
        Run(
            id='run_p8szAQiAIcm3yd3zvXnpo2Bl',
            assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
            cancelled_at=None,
            completed_at=1703570062,
            created_at=1703570057,
            expires_at=None,
            failed_at=None,
            file_ids=[],
            instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
            last_error=None,
            metadata={},
            model='gpt-3.5-turbo-1106',
            object='thread.run',
            required_action=None,
            started_at=1703570058,
            status='completed',
            thread_id='thread_KtqImOv443wCrhGngaLblots',
            tools=[]
        ),
        Run(
            id='run_zxEUhKJrJNJMQk10w9xx2MmV',
            assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
            cancelled_at=None,
            completed_at=1703570056,
            created_at=1703570052,
            expires_at=None,
            failed_at=None,
            file_ids=[],
            instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
            last_error=None,
            metadata={},
            model='gpt-3.5-turbo-1106',
            object='thread.run',
            required_action=None,
            started_at=1703570053,
            status='completed',
            thread_id='thread_KtqImOv443wCrhGngaLblots',
            tools=[]
        )
    ],
    object='list',
    first_id='run_oNEohwsNB800aPqcX0XmzrV0',
    last_id='run_zxEUhKJrJNJMQk10w9xx2MmV',
    has_more=False
)

### 刪除討論串

In [ ]:
response = client.beta.threads.delete(thread_id=new_thread.id)
pprint(response)

ThreadDeleted(id='thread_KtqImOv443wCrhGngaLblots', deleted=True, object='thread.deleted')

### 上傳文件增添知識庫

In [ ]:
!curl -L "https://flagtech.github.io/F4762/Hello.py" -o "Hello.py"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   894  100   894    0     0   5146      0 --:--:-- --:--:-- --:--:--  5137


In [ ]:
file = client.files.create(
  file=open("Hello.py", "rb"),
  purpose="assistants"
)

### 修改助理的檔案清單設定

In [ ]:
assistant_file = client.beta.assistants.update(
    assistant.id,
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)
pprint(assistant_file)

Assistant(
    id='asst_EHR4omyzzojIfQ8spmohkvaB',
    created_at=1703569838,
    description=None,
    file_ids=['file-Q0qnb55P5WATrtbvhMGxcpJS'],
    instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
    metadata={},
    model='gpt-3.5-turbo-1106',
    name='運動達人',
    object='assistant',
    tools=[ToolRetrieval(type='retrieval')]
)

In [ ]:
assistant_file = client.beta.assistants.files.retrieve(
  assistant_id=assistant.id,
  file_id=file.id
)
pprint(assistant_file)

AssistantFile(
    id='file-Q0qnb55P5WATrtbvhMGxcpJS',
    assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
    created_at=1703570087,
    object='assistant.file'
)

In [ ]:
assistant_files = client.beta.assistants.files.list(
  assistant_id=assistant.id
)
pprint(assistant_files)

SyncCursorPage[AssistantFile](
    data=[
        AssistantFile(
            id='file-Q0qnb55P5WATrtbvhMGxcpJS',
            assistant_id='asst_EHR4omyzzojIfQ8spmohkvaB',
            created_at=1703570087,
            object='assistant.file'
        )
    ],
    object='list',
    first_id='file-Q0qnb55P5WATrtbvhMGxcpJS',
    last_id='file-Q0qnb55P5WATrtbvhMGxcpJS',
    has_more=False
)

In [ ]:
deleted_assistant_file = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file.id
)
pprint(deleted_assistant_file)

FileDeleteResponse(id='file-Q0qnb55P5WATrtbvhMGxcpJS', deleted=True, object='assistant.file.deleted')

### 刪除文件

In [ ]:
file_list = client.files.list(
    purpose='assistants'
)
for i in file_list:
    pprint(i)

In [ ]:
deleted_file = client.files.delete(file_list.data[0].id)
pprint(deleted_file)

IndexError: ignored

### 列出所有助理

In [ ]:
my_assistants = client.beta.assistants.list(
    limit = 1
)
for i in my_assistants.data:
    pprint(i)
    response = client.beta.assistants.delete(i.id)
    pprint(response)

Assistant(
    id='asst_EHR4omyzzojIfQ8spmohkvaB',
    created_at=1703569838,
    description=None,
    file_ids=[],
    instructions='你是一位熱愛運動的達人, 具有各種與運動相關的知識',
    metadata={},
    model='gpt-3.5-turbo-1106',
    name='運動達人',
    object='assistant',
    tools=[ToolRetrieval(type='retrieval')]
)

AssistantDeleted(id='asst_EHR4omyzzojIfQ8spmohkvaB', deleted=True, object='assistant.deleted')

## 8-3 使用內建工具

### Code interpreter

In [ ]:
assistant = client.beta.assistants.create(
    name="網頁設計助理",
    instructions="你是一位網頁設計師, 能依照對話內容設計和修改出網頁, "
                 "請生成HTML檔",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106",
)
assistant_id = assistant.id
pprint(assistant)

Assistant(
    id='asst_jRwS6xuOVrBr8SX6ALwNcdod',
    created_at=1703570178,
    description=None,
    file_ids=[],
    instructions='你是一位網頁設計師, 能依照對話內容設計和修改出網頁, 請生成HTML檔',
    metadata={},
    model='gpt-3.5-turbo-1106',
    name='網頁設計助理',
    object='assistant',
    tools=[ToolCodeInterpreter(type='code_interpreter')]
)

In [ ]:
thread = client.beta.threads.create()
thread_id = thread.id
pprint(thread)

Thread(id='thread_HVlKuweQ5Ryug70ePoPqN65R', created_at=1703570221, metadata={}, object='thread')

In [ ]:
message, run = input_and_run("請幫我製作一個精美的天氣預報網頁, "
                             "先設置相關欄位, 有地點、日期、溫度和天氣狀況, "
                             "其他設定由你決定。",
                             thread_id, assistant_id)

run = wait_on_run(run)

messages = client.beta.threads.messages.list(
    thread_id=thread_id, order="asc", after=message.id)
for part in messages:
    print(f'訊息識別碼：{part.id}')
    print(f'AI 回覆：{part.content[0].text.value}')

訊息識別碼：msg_rLFIzt9Eut1DJEcRg1Wh0nfy
AI 回覆：好的，我們可以加上一些圖示和圖表來使網頁更生動。讓我們先設計一個基本的網頁，包含地點、日期、溫度和天氣狀況的欄位。接著再加入圖示和圖表。

請稍等，我會開始設計網頁。
訊息識別碼：msg_yMf1lNmc1G6HLozV5yF7MWqy
AI 回覆：網頁已經設計完成並儲存為HTML檔案。您可以從以下連結下載：

[weather_forecast.html](sandbox:/mnt/data/weather_forecast.html)

請下載檔案並在瀏覽器中開啟，檢查是否符合您的期望。接下來，如果有需要進行修改或添加其他功能，請告訴我。


In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread_id,
    run_id=run.id,
    order="asc"
)
pprint(run_steps)

SyncCursorPage[RunStep](
    data=[
        RunStep(
            id='step_qE5FZ2bYtyn2lldxRsa7LKw7',
            assistant_id='asst_jRwS6xuOVrBr8SX6ALwNcdod',
            cancelled_at=None,
            completed_at=1703570227,
            created_at=1703570224,
            expired_at=None,
            failed_at=None,
            last_error=None,
            metadata=None,
            object='thread.run.step',
            run_id='run_KzBPRWBji8fBP0TSpffa4Y44',
            status='completed',
            step_details=MessageCreationStepDetails(
                message_creation=MessageCreation(message_id='msg_rLFIzt9Eut1DJEcRg1Wh0nfy'),
                type='message_creation'
            ),
            thread_id='thread_HVlKuweQ5Ryug70ePoPqN65R',
            type='message_creation',
            expires_at=None
        ),
        RunStep(
            id='step_NTfW0jpqSpgqRPjDaVVyYSqi',
            assistant_id='asst_jRwS6xuOVrBr8SX6ALwNcdod',
            cancelled_at=None,
            completed_at=1703570236,
            created_at=1703570227,
            expired_at=None,
            failed_at=None,
            last_error=None,
            metadata=None,
            object='thread.run.step',
            run_id='run_KzBPRWBji8fBP0TSpffa4Y44',
            status='completed',
            step_details=ToolCallsStepDetails(
                tool_calls=[
                    CodeToolCall(
                        id='call_GGRVuCB3B0pqpMVtQfsZB4D7',
                        code_interpreter=CodeInterpreter(
                            input='# 產生基本的HTML模板\r\nhtml_content = """\r\n<!DOCTYPE html>\r\n<html 
lang="en">\r\n<head>\r\n    <meta charset="UTF-8">\r\n    <meta name="viewport" content="width=device-width, 
initial-scale=1.0">\r\n    <title>天氣預報</title>\r\n    <style>\r\n        body {\r\n            font-family: 
Arial, sans-serif;\r\n            background-color: #f4f4f4;\r\n            margin: 0;\r\n            padding: 
0;\r\n        }\r\n        header {\r\n            background-color: #007bff;\r\n            color: white;\r\n     
padding: 1em;\r\n            text-align: center;\r\n            font-size: 1.5em;\r\n        }\r\n        
.container {\r\n            max-width: 800px;\r\n            margin: 20px auto;\r\n            padding: 20px;\r\n  
background-color: white;\r\n            border-radius: 5px;\r\n            box-shadow: 0 0 10px rgba(0, 0, 0, 
0.1);\r\n        }\r\n        table {\r\n            width: 100%;\r\n            border-collapse: collapse;\r\n    
margin-bottom: 20px;\r\n        }\r\n        th, td {\r\n            padding: 12px 15px;\r\n            text-align:
left;\r\n        }\r\n        th {\r\n            background-color: #007bff;\r\n            color: white;\r\n      
}\r\n    </style>\r\n</head>\r\n<body>\r\n    <header>天氣預報</header>\r\n    <div class="container">\r\n        
<h2>地點: 台北市</h2>\r\n        <table>\r\n            <tr>\r\n                <th>日期</th>\r\n                
<th>溫度</th>\r\n                <th>天氣狀況</th>\r\n            </tr>\r\n            <tr>\r\n                
<td>10/01/2022</td>\r\n                <td>28°C</td>\r\n                <td>晴</td>\r\n            </tr>\r\n       
<tr>\r\n                <td>10/02/2022</td>\r\n                <td>29°C</td>\r\n                <td>多雲</td>\r\n  
</tr>\r\n        </table>\r\n    </div>\r\n</body>\r\n</html>\r\n"""\r\n\r\n# 將HTML內容存為檔案\r\nfile_path = 
\'/mnt/data/weather_forecast.html\'\r\nwith open(file_path, \'w\') as file:\r\n    
file.write(html_content)\r\n\r\nfile_path',
                            outputs=[
                                CodeInterpreterOutputLogs(logs="'/mnt/data/weather_forecast.html'", type='logs')
                            ]
                        ),
                        type='code_interpreter'
                    )
                ],
                type='tool_calls'
            ),
            thread_id='thread_HVlKuweQ5Ryug70ePoPqN65R',
            type='tool_calls',
            expires_at=None
    

In [ ]:
run_step = client.beta.threads.runs.steps.retrieve(
    thread_id=thread_id,
    run_id=run.id,
    step_id=run_steps.data[1].id
)
pprint(run_step)

RunStep(
    id='step_NTfW0jpqSpgqRPjDaVVyYSqi',
    assistant_id='asst_jRwS6xuOVrBr8SX6ALwNcdod',
    cancelled_at=None,
    completed_at=1703570236,
    created_at=1703570227,
    expired_at=None,
    failed_at=None,
    last_error=None,
    metadata=None,
    object='thread.run.step',
    run_id='run_KzBPRWBji8fBP0TSpffa4Y44',
    status='completed',
    step_details=ToolCallsStepDetails(
        tool_calls=[
            CodeToolCall(
                id='call_GGRVuCB3B0pqpMVtQfsZB4D7',
                code_interpreter=CodeInterpreter(
                    input='# 產生基本的HTML模板\r\nhtml_content = """\r\n<!DOCTYPE html>\r\n<html 
lang="en">\r\n<head>\r\n    <meta charset="UTF-8">\r\n    <meta name="viewport" content="width=device-width, 
initial-scale=1.0">\r\n    <title>天氣預報</title>\r\n    <style>\r\n        body {\r\n            font-family: 
Arial, sans-serif;\r\n            background-color: #f4f4f4;\r\n            margin: 0;\r\n            padding: 
0;\r\n        }\r\n        header {\r\n            background-color: #007bff;\r\n            color: white;\r\n     
padding: 1em;\r\n            text-align: center;\r\n            font-size: 1.5em;\r\n        }\r\n        
.container {\r\n            max-width: 800px;\r\n            margin: 20px auto;\r\n            padding: 20px;\r\n  
background-color: white;\r\n            border-radius: 5px;\r\n            box-shadow: 0 0 10px rgba(0, 0, 0, 
0.1);\r\n        }\r\n        table {\r\n            width: 100%;\r\n            border-collapse: collapse;\r\n    
margin-bottom: 20px;\r\n        }\r\n        th, td {\r\n            padding: 12px 15px;\r\n            text-align:
left;\r\n        }\r\n        th {\r\n            background-color: #007bff;\r\n            color: white;\r\n      
}\r\n    </style>\r\n</head>\r\n<body>\r\n    <header>天氣預報</header>\r\n    <div class="container">\r\n        
<h2>地點: 台北市</h2>\r\n        <table>\r\n            <tr>\r\n                <th>日期</th>\r\n                
<th>溫度</th>\r\n                <th>天氣狀況</th>\r\n            </tr>\r\n            <tr>\r\n                
<td>10/01/2022</td>\r\n                <td>28°C</td>\r\n                <td>晴</td>\r\n            </tr>\r\n       
<tr>\r\n                <td>10/02/2022</td>\r\n                <td>29°C</td>\r\n                <td>多雲</td>\r\n  
</tr>\r\n        </table>\r\n    </div>\r\n</body>\r\n</html>\r\n"""\r\n\r\n# 將HTML內容存為檔案\r\nfile_path = 
\'/mnt/data/weather_forecast.html\'\r\nwith open(file_path, \'w\') as file:\r\n    
file.write(html_content)\r\n\r\nfile_path',
                    outputs=[CodeInterpreterOutputLogs(logs="'/mnt/data/weather_forecast.html'", type='logs')]
                ),
                type='code_interpreter'
            )
        ],
        type='tool_calls'
    ),
    thread_id='thread_HVlKuweQ5Ryug70ePoPqN65R',
    type='tool_calls',
    expires_at=None
)

In [ ]:
pprint(messages.data[-1])

ThreadMessage(
    id='msg_yMf1lNmc1G6HLozV5yF7MWqy',
    assistant_id='asst_jRwS6xuOVrBr8SX6ALwNcdod',
    content=[
        MessageContentText(
            text=Text(
                annotations=[
                    TextAnnotationFilePath(
                        end_index=95,
                        file_path=TextAnnotationFilePathFilePath(file_id='file-qmlalTBl2yCmGG17UsqhKpRw'),
                        start_index=56,
                        text='sandbox:/mnt/data/weather_forecast.html',
                        type='file_path'
                    )
                ],
                value='網頁已經設計完成並儲存為HTML檔案。您可以從以下連結下載：\n\n[weather_forecast.html](sandbox:
/mnt/data/weather_forecast.html)\n\n請下載檔案並在瀏覽器中開啟，檢查是否符合您的期望。接下來，如果有需要進行修改或
添加其他功能，請告訴我。'
            ),
            type='text'
        )
    ],
    created_at=1703570236,
    file_ids=['file-qmlalTBl2yCmGG17UsqhKpRw'],
    metadata={},
    object='thread.message',
    role='assistant',
    run_id='run_KzBPRWBji8fBP0TSpffa4Y44',
    thread_id='thread_HVlKuweQ5Ryug70ePoPqN65R'
)

In [ ]:
from IPython.display import HTML

def show_html(messages):
    for i in messages.data:
        # 找出有文件內容的對話物件
        if i.content[0].text.annotations:
            file_index =  i.content[0].text.annotations
            # 找到文件識別碼
            file_ids = file_index[0].file_path.file_id
            content = client.files.content(file_ids)
            # 儲存 HTML
            content.stream_to_file('test.html')
            # 顯示 HTML
            html_content = content.content.decode('utf-8')
            display(HTML(html_content))

show_html(messages)

日期,溫度,天氣狀況
10/01/2022,28°C,晴
10/02/2022,29°C,多雲


In [ ]:
while True:
    question = input('請輸入要求：')
    if not question.strip():
        break
    message, run = input_and_run(question, thread_id, assistant_id)

    run = wait_on_run(run)

    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread_id,order="asc", after=message.id)
        for part in messages:
            print(f'AI 回覆：{part.content[0].text.value}')
        show_html(messages)
    else:
        print(run.status)

請輸入要求：請加入背景
AI 回覆：當然，我們可以加入一個動態的背景圖片來提升網頁的吸引力。讓我們立即加入背景圖片，請稍等片刻。
AI 回覆：背景圖片已成功加入並且網頁內容已經更新。您可以從以下連結下載新版本的HTML檔案，並在瀏覽器中開啟以檢查。 

[weather_forecast_with_bg.html](sandbox:/mnt/data/weather_forecast_with_bg.html)


日期,溫度,天氣狀況
10/01/2022,28°C,晴
10/02/2022,29°C,多雲


請輸入要求：增加下拉式表單
AI 回覆：理解了，您想加入一個下拉式表單供使用者選擇不同的地點以查詢天氣預報。讓我立即修改網頁，請稍等。
AI 回覆：下拉式表單已經成功加入並且網頁內容已經更新。您可以從以下連結下載新版本的HTML檔案，並在瀏覽器中開啟以檢查。

[weather_forecast_with_dropdown.html](sandbox:/mnt/data/weather_forecast_with_dropdown.html)


日期,溫度,天氣狀況
10/01/2022,28°C,晴
10/02/2022,29°C,多雲


請輸入要求：


### Retrieval 檢索器 - RAG


In [ ]:
!mkdir "旅遊"
!curl -L "https://ppt.cc/fscClx" -o "旅遊/food.pdf"
!curl -L "https://ppt.cc/f2BqBx" -o "旅遊/history.pdf"
!curl -L "https://ppt.cc/fuarUx" -o "旅遊/nature.pdf"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3576k  100 3576k    0     0  2139k      0  0:00:01  0:00:01 --:--:-- 17.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3357k  100 3357k    0     0  2429k      0  0:00:01  0:00:01 --:--:-- 2429k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3803k  100 3803k    0     0  2758k      0  0:00:01  0:00:01 --:--:-- 2758k


In [ ]:
import os
ids = []
allfilelist = os.listdir('/content/旅遊')
for path in allfilelist:
    file = client.files.create(
        file=open(f"/content/旅遊/{path}", "rb"),
        purpose='assistants'
    )
    ids.append(file.id)

In [ ]:
assistant = client.beta.assistants.create(
    name="旅遊助理",
    instructions="你是一位旅遊規劃者, 可以根據旅遊文件規劃旅遊行程",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=ids
)
assistant_id = assistant.id
pprint(assistant)

thread = client.beta.threads.create()
thread_id = thread.id

Assistant(
    id='asst_TNZ60IGVpv2oBNtQdLz0ViKt',
    created_at=1703570337,
    description=None,
    file_ids=['file-7gOt18kaBgxzZw8zlqbp33Yc', 'file-BhyqYnwfjovD9DkC5bfGRBuX', 'file-q7xrzBOA27eF0k8iAIpnKGJR'],
    instructions='你是一位旅遊規劃者, 可以根據旅遊文件規劃旅遊行程',
    metadata={},
    model='gpt-4-1106-preview',
    name='旅遊助理',
    object='assistant',
    tools=[ToolRetrieval(type='retrieval')]
)

In [ ]:
from IPython.display import Markdown

def output_text(thread_id, message):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id, order="asc", after=message.id)
    for message in messages:
        display(Markdown(message.content[0].text.value))

In [ ]:
message, run = input_and_run("請規劃台中一日遊行程",
                             thread_id, assistant.id)
run = wait_on_run(run)
pprint(run.status)
output_text(thread_id, message)

completed

基於提供的資料，我為你規劃了一個台中一日遊行程，以下是建議的旅遊景點及順序：

1. **彩虹眷村**【13†source】: 推薦先從富有色彩和歷史意義的彩虹眷村開始你的一日遊。

2. **霧峰林家宮保第園區**【14†source】: 隨後參觀一下霧峰林家宮保第園區，體驗尊貴的清朝臺灣第一官宅氣勢。

3. **亞洲大學現代美術館**【15†source】: 在霧峰區附近，你還可以順遊亞洲大學的現代美術館，感受國際大師安藤忠雄氛圍。

4. **摘星山莊**【16†source】: 繼而前往摘星山莊，一睹被譽為「臺灣十大民宅之首」的豪宅之美。

5. **筱雲山莊**【21†source】: 筱雲山莊同樣為百年官宅建築，感受保存良好的古蹟風貌。

6. **東勢客家文化園區**【24†source】: 最後到東勢客家文化園區，體驗客家文化和山城的美。

請注意開放時間及休息日，以免前往時遇到關閉的情況。希望這個行程能為你的台中之旅增添樂趣！有其他特別需求或想要更多建議，請告訴我。

In [ ]:
!curl -L "https://ppt.cc/fAzoVx" -o "Travel_In_Taoyuan.json"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  119k  100  119k    0     0  85758      0  0:00:01  0:00:01 --:--:-- 85758


In [ ]:
file = client.files.create(
  file=open("/content/Travel_In_Taoyuan.json", "rb"),
  purpose='assistants'
)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="可以規劃去桃園兩天一夜的行程嗎?",
    file_ids=[file.id]
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant.id,
)
run = wait_on_run(run)
pprint(run.status)
output_text(thread_id, message)

completed

根据提供的资料，为您规划了一个桃园两天一夜的行程如下：

### 第一天:
- 早餐后开始您的旅程
- **新北市立黄金博物馤**：首先参观黄金博物馆，了解这里的黄金文化历史。
- **黄金神社**：参观黄金神社，感受其特有的宁静和历史韵味。
- **无耳茶壶山登山步道**：接着前往无耳茶壶山，进行一个轻松的登山之旅。
- **报时山观景台**：午间在报时山观景台欣赏美景，并在此用午餐。
- 返回**黄金神社**：下午返回黄金神社，进一步探索。
- **基隆廟口夜市+晚餐**：傍晚，前往基隆廟口夜市品尝当地的美食。
- **基隆旅宿**：晚餐后，前往基隆旅宿休息。

### 第二天:
- 早餐后退房准备出发
- 参观**大漢溪山豬湖生態親水園區**：从中壢后火车站出发，去大漢溪的山豬湖生態親水园区。
- **李鄭芳古厝**：继续探访李鄭芳古厝，领略古厝的精美雕刻和建筑美学。
- 特色午餐：在**香满园花园餐厅**或**台湾砖窯鸡**或**大溪山水庭园餐厅**享用特色餐。

以上行程需要根据实际开放时间和交通安排进行微调，同时建议早点开始一天的行程以避免错过任何景点。使用北北基桃好玩卡可以享受相关景点的优惠【30†source】。祝您在桃园的两天时光玩得开心！

如果要台中與桃園各玩一天, 該如何規劃?

我想要去台中進行一趟美食之旅

In [ ]:
while True:
    question = input('請輸入要求：')
    if not question.strip():
        break
    message, run = input_and_run(question, thread_id, assistant_id)

    run = wait_on_run(run)
    pprint(run.status)
    output_text(thread_id, message)

請輸入要求：如果要台中與桃園各玩一天, 該如何規劃?


completed

當然可以。根據您的需求，這裡是一個簡單的行程規劃，台中與桃園各玩一天：

### 台中一日遊行程
- **上午**
  - **彩虹眷村**：開始您的台中之旅，參觀充滿藝術和歷史的彩虹眷村【11†source】。
  - **霧峰林家宮保第園區**：接著遊覽霧峰林家宮保第園區，感受清朝臺灣第一官宅的氣勢【12†source】。
  - **午餐**：在附近享用道地台中美食。
- **下午**
  - **亞洲大學現代美術館**：體驗國際大師安藤忠雄設計的亞洲大學現代美術館【13†source】。
  - **摘星山莊**：參觀被譽為「臺灣十大民宅之首」的摘星山莊【14†source】。
- **晚上**
  - **台中公園（湖心亭）**：在台中公園輕鬆散步，享受美麗的湖心亭景色【18†source】。
  - **晚餐**：品嘗台中當地特色晚餐。

### 桃園一日遊行程
- **上午**
  - **大漢溪山豬湖生態親水園區**：前往大漢溪欣賞當地的自然景觀與生態【30†source】。
  - **李騰芳古厝**：探訪歷史悠久的李騰芳古厝，體驗百年建築的魅力【30†source】。
  - **午餐**：選擇在香滿園花園餐廳、台灣磚窯雞或大溪山水庭園餐廳用餐【30†source】。
- **下午**
  - **桃園其他景點**：可根據您的興趣，選擇參觀其他桃園的景點，例如大溪老街、角板山、藝文特區等。
  - **晚餐**：在桃園品嚐道地的晚餐，享受桃園的夜生活。

這只是一個基本的行程建議，您可以根據自己的喜好和時間進行適當調整。請記得考慮各個旅遊景點的開放時間和所需的交通時間。祝您旅途愉快！

請輸入要求：我想要去台中進行一趟美食之旅


completed

很抱歉，似乎在操作文檔時發生了一些錯誤，我無法從您提供的文件中獲得台中的美食資訊。我可以重新嘗試獲取有關台中美食的資料，或者我可以根據以往的知識給您一些建議。您希望我如何協助您？

請輸入要求：


In [ ]:
new_thread = client.beta.threads.create()
new_thread_id = new_thread.id

while True:
    question = input('請輸入要求：')
    if not question.strip():
        break
    message, run = input_and_run(question, new_thread_id,
                                 assistant_id)
    run = wait_on_run(run)
    pprint(run.status)
    output_text(new_thread_id, message)

請輸入要求：可以規劃去桃園兩天一夜的行程嗎?


completed

根據提供的資訊，我為您規劃了兩天一夜遊覽桃園的行程。

### 第一天：
1. **臺中火車站 (舊站)** - 這是全臺唯一三代同堂的車站，您可以欣賞到由日式木造小站、巴洛克式建築舊站，到未來設計感新站，這樣跨時空的美景【14†source】。

2. **文化部文化資產園區** - 這裡是遺址保存最完整的臺灣酒廠，現已打造成藝文特區，古蹟和藝術交織在一起【14†source】。

3. **臺中州廳** - 來這裡可觀賞百年前總統府御用建築師的作品，建築以法式風格為主，您可以體驗到細膩的古建築風味【14†source】。

4. **臺中刑務所演武場** - 這是日據時期留下的珍貴武道場，充滿日式禪風，相當適合靜下心來參觀【14†source】。

### 第二天：
5. **臺中公園（湖心亭）** - 臺灣首座百年地標公園，您可以選擇划船或是散步在園內，體驗典型的台中風情【14†source】。

6. **柳川水岸步道** - 最後您可以在這條生態工法開發的景觀河岸散步，享受一份悠閒【14†source】。

請根據這些景點的開放時間安排旅行的行程，希望這個安排能讓您在桃園的兩天一夜之旅中擁有美好的體驗！如果您需要住宿或餐飲建議，請告知我，我可以繼續為您規劃。

請輸入要求：我是要去桃園不是台中


completed

對不起，我之前提供的資訊是有關臺中而非桃園的景點，我會重新進行搜索和規劃。

經過查閱目前提供的旅遊文件，似乎沒有特定的提及桃園兩天一夜的詳細行程。為了制定一個合適的行程，我將使用已知的資訊以及一些常見的觀光推薦，來為您規劃桃園的旅遊。

### 桃園兩天一夜行程建議：

#### 第一天：
- **搭車前往桃園** - 確保您選擇的交通方式能在早上抵達桃園，以便有更多的觀光時間。
- **登入窯文化園區** - 您可以參觀這個結合了文化與藝術的園區，了解桃園當地的手工藝以及傳統窯燒技藝。
- **中壢夜市** - 晚上可以前往中壢夜市，享受當地的特色小吃，體驗台灣夜市的熱鬧氛圍。
- **入住飯店** - 選擇一家舒適的飯店進行休息，以備次日的活動。

#### 第二天：
- **大溪老街** - 逛逛大溪老街，品嘗當地美食，欣賞老街上的傳統建築。
- **角板山** - 前往角板山賞花遊憩區，你可以在此處遠眺大漢溪的美景，或是體驗該地區的戶外活動。
- **大園區海邊** - 如果時間允許，可以去大園區的海邊去欣賞海景，也許還能偶遇到一些水上活動。
- **返回** - 下午或傍晚搭乘交通工具返回家，結束兩天一夜的旅行。

如果需要更加具體的行程規劃，例如特定的景點訊息、餐廳推薦或是住宿選擇，請提供更多細節或需求，我會進一步為您做出安排。

請輸入要求：


### Function calling

In [ ]:
from googlesearch import search
def google_res(user_input):
    response = ''
    for i in search(user_input, advanced=True, num_results=5):
        response += f'{i.title}\n{i.description}\n'
    return response

In [ ]:
assistant = client.beta.assistants.create(
  name="搜尋助理",
  instructions="取得 Google 搜尋結果",
  model="gpt-3.5-turbo-1106",
  tools=[
    {"type": "function",
    "function": {
      "name": "google_res",
      "description": "如果問題為你不知道的事情需要網路搜尋, "\
      "才會使用這個搜尋函式, 使用繁體中文作回覆",
      "parameters": {
        "type": "object",
        "properties": {
          "user_input": {"type": "string"}
        },
        "required": ["user_input"]
      }
    }
  }]
)
assistant_id = assistant.id

thread = client.beta.threads.create()
thread_id = thread.id

In [ ]:
message, run = input_and_run("2023年金鐘獎男主角是?",
                             thread_id, assistant_id)
run = wait_on_run(run)
pprint(run)

Run(
    id='run_3WS96K5xKyhE0UJ9oxVtFwoI',
    assistant_id='asst_IH1ib1TQLfhsCtwRtr1KRZM9',
    cancelled_at=None,
    completed_at=None,
    created_at=1703584541,
    expires_at=1703585141,
    failed_at=None,
    file_ids=[],
    instructions='取得 Google 搜尋結果',
    last_error=None,
    metadata={},
    model='gpt-4-1106-preview',
    object='thread.run',
    required_action=RequiredAction(
        submit_tool_outputs=RequiredActionSubmitToolOutputs(
            tool_calls=[
                RequiredActionFunctionToolCall(
                    id='call_cX5hqA8MtQ8yVLueBXfycVrd',
                    function=Function(
                        arguments='{"user_input":"2023 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n踴 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 譯 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n 譯 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n 寅 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n獎男主角"}',
                        name='google_res'
                    ),
                    type='function'
                )
            ]
        ),
        type='submit_tool_outputs'
    ),
    started_at=1703584542,
    status='requires_action',
    thread_id='thread_yWmunltK9QKaGB8gZjX2ISOw',
    tools=[
        ToolAssistantToolsFunction(
            function=FunctionDefinition(
                name='google_res',
                description='如果問題為你不知道的事情需要網路搜尋, 才會使用這個搜尋函式, 使用繁體中文作回覆',
                parameters={
                    'type': 'object',
                    'properties': {'user_input': {'type': 'string'}},
                    'required': ['user_input']
                }
            ),
            type='function'
        )
    ]
)

In [ ]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
pprint(tool_calls)

[
    RequiredActionFunctionToolCall(
        id='call_cX5hqA8MtQ8yVLueBXfycVrd',
        function=Function(
            arguments='{"user_input":"2023 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n踴 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n \\n \\n 譯 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n
\\n \\n 譯 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n 
\\n \\n \\n \\n 寅 \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n獎男主角"}',
            name='google_res'
        ),
        type='function'
    )
]

In [ ]:
import json
# 函式識別碼、名稱和參數
tool_id = tool_calls[0].id
func_name = tool_calls[0].function.name
func_args = json.loads(tool_calls[0].function.arguments)

print(f'識別碼：{tool_id}\n函式名稱：{func_name}\n參數：{func_args}')

# 將參數代入函式中
output = google_res(**func_args)

識別碼：call_cX5hqA8MtQ8yVLueBXfycVrd
函式名稱：google_res
參數：{'user_input': '2023 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n踴 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n 譯 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n 譯 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n 

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread_id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": tool_id,
        "output": output,
      }
    ]
)
run = wait_on_run(run)
output_text(thread_id, message)

很抱歉，我無法找到具體的資料來確認誰是2023年金鐘獎的男主角。建議您可以查看官方公告或娛樂新聞以獲得最準確的信息。

In [ ]:
def tool_outputs(tool_calls):
    outputs = []
    for tool in tool_calls:
        func_name = tool.function.name
        func_args = json.loads(tool.function.arguments)
        print(f'{func_name}({func_args})')
        reponse = google_res(**func_args)
        outputs.append({
            'tool_call_id': tool.id,
            'output': reponse
        })
    return outputs

亞洲空汙最嚴重的地區是?那台灣呢?

2023年台灣金馬獎影帝和金曲獎歌王分別是?

In [ ]:
while True:
    question = input('請輸入要求：')
    if not question.strip():
        break
    message, run = input_and_run(question, thread_id, assistant_id)

    run = wait_on_run(run)

    if run.status == 'requires_action':
        pprint('搜尋中...')
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        outputs = tool_outputs(tool_calls)
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread_id,
            run_id=run.id,
            tool_outputs=outputs
            )
        run = wait_on_run(run)
        output_text(thread_id, message)
    else:
        output_text(thread_id, message)

請輸入要求：亞洲空汙最嚴重的地區是?那台灣呢?


搜尋中...

google_res({'user_input': '亞洲空汙最嚴重的地區是'})
google_res({'user_input': '台灣空汙情況'})


亞洲空汙最嚴重的地區包括印度、中國、巴基斯坦、伊拉克等地，其中印度包括了八個城市在亞洲空氣污染排名中佔據前十名。

而台灣的空氣污染情況可以通過Air Quality Index (AQI)等數據來評估，根據最近的報告，台灣的PM2.5平均值為13.4 μg/m³，優於前一年的16.2 μg/m³，顯示了空氣品質的改善。

請輸入要求：2023年台灣金馬獎影帝和金曲獎歌王分別是?


搜尋中...

google_res({'user_input': '2023年台灣金馬獎影帝是'})
google_res({'user_input': '2023年台灣金曲獎歌王是'})


2023年台灣金馬獎影帝是吳慷仁，而金曲獎歌王則是由A-Lin榮獲。

請輸入要求：
